In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, make_scorer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

# from sklearn.feature_selection import SelectKBest, f_regression

In [3]:
np.random.seed(7)

# metrics

In [4]:
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return (f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def score(y_val, pred):
    f_value = fscore(y_val, pred)
    mae_value = maeOverFscore(y_val, pred)
    
    print(f"fscore : {f_value} \t maeOverFscore : {mae_value}")
    
    return f_value, mae_value

# fscore_sklearn = make_scorer(fscore)
# maeOverFscore_sklearn = make_scorer(maeOverFscore)

# data load

In [5]:
base = os.getcwd()
data_path = os.path.join(base, 'data')
submit_path = os.path.join(base, 'submit')

def load_data(name):
    return np.load(os.path.join(data_path, f"{name}.npy"))

def load(name):
    if name == "test" :
        return load_data('x', 'test')
    return (load_data(f'x_{name}'), load_data(f'y_{name}'))

def reshape(data):
    return data.reshape(data.shape[0] * 40 * 40, data.shape[-1])

In [6]:
data = load_data('EDA')

# Feature Selection

## select K
![img](feacture_selection.PNG)

## selectK 7
- [False, False,  True,  True,  True,  True,  True,  True, False, False, False, False, False, True]

## selectK 8
- [False, False,  True,  True,  True,  True,  True,  True, False, False, False, True, False, True]

In [7]:
selectK_7 = [2, 3, 4, 5, 6, 7, 13]
selectK_8 = [2, 3, 4, 5, 6, 7, 11, 13]

# Seperate Data

In [8]:
X = data[:, selectK_8]
Y = data[:,  -1]

del data
print(X.shape, Y.shape)

(121608234, 8) (121608234,)


# Ridge + KFold

In [12]:
def train(kfold, clf, scaler=None):
    for (train_idx, val_idx) in kfold.split(Y):
        x = X[train_idx, :]
        
        if scaler is not None:
            x = scaler.fit_transform(x)

        ridge = clf.fit(x, Y[train_idx])

        del x
        x = X[val_idx, :]

        if scaler is not None:
            x = scaler.transform(x)
        
        pred = clf.predict(x)
        score(Y[val_idx], pred)

        del x

In [ ]:
kfold = KFold(n_splits=4, random_state=7, shuffle=True)
clf = RandomForestRegressor(n_estimators=100, random_state=7)
# clf = Ridge(alpha=10.0)
# sclaer= PCA(n_components=2, random_state=7)

train(kfold, clf)

# Ridge alpha 12.5 SelectK 8
- fscore : 0.25464065823329085 	 maeOverFscore : 6.752701127612951
- fscore : 0.25458534451407944 	 maeOverFscore : 6.7528914220204515
- fscore : 0.2545837754161434 	 maeOverFscore : 6.702930965930638
- fscore : 0.25427823328070515 	 maeOverFscore : 6.747219104960767

# Ridge alpha 10.0 SelectK 8
- fscore : 0.25464065823329085 	 maeOverFscore : 6.752701129278926
- fscore : 0.25458534451407944 	 maeOverFscore : 6.75289142477857
- fscore : 0.2545837754161434 	 maeOverFscore : 6.70293096792134
- fscore : 0.25427826739578746 	 maeOverFscore : 6.747218201741558

# Ridge alpha 10.0 SelectK 8 StandardScaler
- fscore : 0.2546402018844707 	 maeOverFscore : 6.752709268799811
- fscore : 0.2545845515075978 	 maeOverFscore : 6.752907726649149
- fscore : 0.2545843704403055 	 maeOverFscore : 6.702911044140701
- fscore : 0.25427924623684933 	 maeOverFscore : 6.7471879931893675

# Ridge alpha 10.0 SelectK 8 RobustScaler


# Ridge alpha 10.0 SelectK 8 PCA
- fscore : 0.24107474262012732 	 maeOverFscore : 8.224389019645066
- fscore : 0.2410970325797045 	 maeOverFscore : 8.222939301354975
- fscore : 0.24133791359125045 	 maeOverFscore : 8.159670908854318
- fscore : 0.2408981513939592 	 maeOverFscore : 8.21528441818803

# RandomForestRegressor SelectK



# Submit

In [11]:
def submit(clf, name, preprocess=None):
    x_test = load('test')
    x_test = reshape(x_test)
    
    if preprocess is not None:
        x_test = preprocess.transform(x_test)
        print("transform")
    
    pred = clf.predict(x_test)

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index = False)

In [20]:
submit(ridge, 'new_Ridge_SelectK', selectK)

transform


- https://dacon.io/competitions/official/235591/mysubmission/
- D:\인공지능_공모전\github\submit